<a href="https://colab.research.google.com/github/bartala/PCLvsCAPS/blob/main/Reduced_PCL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from io import StringIO
from sklearn.metrics import roc_curve, auc


PTH = '/content/data'

In [ ]:
df = pd.read_excel(f'{PTH}/PCL_CAPS_ALL.xlsx')
df = df.iloc[:,2:23]
df['PCL_total'] = df.iloc[:,0:20].sum(axis=1)
df['PCL_reduced_total'] = df[["pcl_q10","pcl_q2","pcl_q7","pcl_q15","pcl_q4","pcl_q9"]].sum(axis=1)
df

,pcl_q1,pcl_q2,pcl_q3,pcl_q4,pcl_q5,pcl_q6,pcl_q7,pcl_q8,pcl_q9,pcl_q10,...,pcl_q14,pcl_q15,pcl_q16,pcl_q17,pcl_q18,pcl_q19,pcl_q20,caps_ptsd1,PCL_total,PCL_reduced_total
0,1,1,1,1,0,2,1,2,0,1,...,0,1,0,3,0,1,2,1,17,5
1,3,2,2,2,2,2,2,1,1,1,...,2,2,0,3,2,1,2,0,34,10
2,1,0,0,0,0,0,0,1,1,0,...,0,1,0,0,0,2,1,0,9,2
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,1
4,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,3,0,0,2,2,3,0,1,0,0,...,0,0,0,3,0,2,2,0,19,2
103,3,3,0,2,1,2,1,0,0,3,...,0,0,0,1,0,1,0,0,22,9
104,2,0,1,1,2,1,1,1,3,3,...,3,2,0,3,3,3,0,1,37,10
105,1,0,0,1,0,2,1,1,1,3,...,0,1,0,2,2,2,0,1,24,7


In [ ]:
# Calc AUC with no ML model

import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve

# CAPS: 0/1 ground truth, PCL5: total score 0–80 (float/int)
CAPS = df['caps_ptsd1']
PCL5 = df['PCL_total'] #PCL_total
#PCL5 = df['PCL_reduced_total']

CAPS = np.asarray(CAPS, dtype=float)
PCL5 = np.asarray(PCL5, dtype=float)

# drop NaNs if any
mask = ~np.isnan(CAPS) & ~np.isnan(PCL5)
y = CAPS[mask]
x = PCL5[mask]

# If higher PCL-5 means more likely PTSD (it does), use as-is:
auc_val = roc_auc_score(y, x)
fpr, tpr, thresholds = roc_curve(y, x)   # for plotting if needed

print("AUC:", auc_val)


AUC: 0.9231481481481482


In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, roc_curve, auc

X = df['PCL_total']
#X = df['PCL_reduced_total']
y = df['caps_ptsd1']

thresholds = range(18, 39)
results = []

for thr in thresholds:
    y_pred = (X >= thr).astype(int)

    # confusion matrix: tn, fp, fn, tp
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()

    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else float('nan')  # recall
    specificity = tn / (tn + fp) if (tn + fp) > 0 else float('nan')
    ppv = tp / (tp + fp) if (tp + fp) > 0 else float('nan')
    npv = tn / (tn + fn) if (tn + fn) > 0 else float('nan')
    j_stat = sensitivity + specificity - 1

    fpr, tpr, _ = roc_curve(y, y_pred)
    roc_auc_thr = auc(fpr, tpr)

    results.append((thr, sensitivity, specificity, ppv, npv, j_stat, roc_auc_thr))

results_df = pd.DataFrame(results, columns=[
    'Threshold', 'Sensitivity', 'Specificity',
    'PPV', 'NPV', 'J_statistic', 'AUC'
])

print(results_df)


    Threshold  Sensitivity  Specificity       PPV       NPV  J_statistic  \
1          19     0.888889       0.7250  0.521739  0.950820     0.613889   
2          20     0.888889       0.7375  0.533333  0.951613     0.626389   
3          21     0.888889       0.7625  0.558140  0.953125     0.651389   
4          22     0.851852       0.8000  0.589744  0.941176     0.651852   
5          23     0.851852       0.8250  0.621622  0.942857     0.676852   
6          24     0.814815       0.8375  0.628571  0.930556     0.652315   
7          25     0.777778       0.8500  0.636364  0.918919     0.627778   
8          26     0.777778       0.8500  0.636364  0.918919     0.627778   
9          27     0.740741       0.8750  0.666667  0.909091     0.615741   
10         28     0.740741       0.8750  0.666667  0.909091     0.615741   
11         29     0.740741       0.9000  0.714286  0.911392     0.640741   
12         30     0.629630       0.9250  0.739130  0.880952     0.554630   
13         3